In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/clay/userdata/irish-maler-cleaned


In [2]:
from pathlib import Path
from pyabc import pyabc

from data_utils import prepare_abc

In [3]:
session_dir = Path('abc_dataset/folk_rnn_abc_key_cleaned')
session_abcs = list(session_dir.rglob('*.abc'))
session_abcs.sort(key=lambda x: int(x.stem))

In [4]:
session_abcs

[PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00001.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00002.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00003.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00004.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00005.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00006.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00007.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00008.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00009.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00010.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00011.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00012.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00013.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00014.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00015.abc'),
 PosixPath('abc_dataset/folk_rnn_abc_key_cleaned/00016.abc'),
 PosixPa

In [5]:
session_tunes, error_list = prepare_abc(session_abcs)

Error in token measure_offset: <Beam "|">
Error in token measure_offset: <Beam "|">
Error in token measure_offset: <Beam "|">
Error in meas_offset_from_repeat_start: <Beam "|">
Error in token measure_offset: <Beam "|">
Error in meas_offset_from_repeat_start: <Tie "-">
Error in token measure_offset: <Beam "|">
Error in meas_offset_from_repeat_start: <Beam ":|">
Error in meas_offset_from_repeat_start: <Beam "|:">
Error in meas_offset_from_repeat_start: <Beam "|:">
Error in meas_offset_from_repeat_start: <Beam "|:">
Error in token measure_offset: <Beam "|">


In [6]:
len(session_tunes), len(error_list), len(session_abcs)

(17601, 4, 17605)

In [7]:
session_tunes[0].header

{'reference number': '1',
 'tune title': "'G Iomain Nan Gamhna",
 'source': 'https://thesession.org/tunes/11931#setting11931',
 'transcription': 'G Major',
 'rhythm': 'slip jig',
 'meter': '9/8',
 'unit note length': '1/8',
 'key': 'C Major'}

In [8]:
title_tunes = [tune for tune in session_tunes if 'x' != tune.header['tune title']]
title_set = set([tune.header['tune title'] for tune in title_tunes])

In [10]:
len(title_tunes), len(title_set)

(14981, 7153)

In [11]:
collect_title = []
collect_tunes = []

for tune in title_tunes:
  if tune.header['tune title'] not in collect_title:
    collect_title.append(tune.header['tune title'])
    collect_tunes.append(tune)

In [14]:
collect_title

["'G Iomain Nan Gamhna",
 'Devil In The Kitchen, The',
 '10th Bat Crossing Rhine',
 'Hoch Lebe Der Mann Mit Dem Hut',
 '1st August',
 '2D Or Not 2D',
 '30 Year, The',
 '30th Anniversary',
 '42 Beech Avenue',
 '5:30am',
 '6-8 Du Petit Sarny',
 "72nd Highlanders' Farewell To Aberdeen, The",
 "Tommy Peoples'",
 "79th's Farewell To Gibraltar",
 '93 Not Out',
 'A Barrel Of Knives',
 'A Bhriogais Uallach',
 'A Birthday',
 'Britches, The',
 'A Bottle Of Vodka, Twenty Marlboro Reds And £50 Cashback, Please',
 'A Breton In Paris',
 'Moll Roe',
 'Love At The Window',
 "A Chur Nan Gobhar As A 'Chreig",
 'Molly On The Shore',
 'A Costa De Galicia',
 'A Country Wedding',
 'A Cup Of Tea',
 'A Cuppa Java',
 'Trip To The Quarry',
 'A Day And An Age',
 'Newfoundland, The',
 'A Deanadh Im',
 'A Dram At The Wake',
 'Fair Wind, The',
 'A Favourite Duet By Mr Holmes',
 'A Fig For A Kiss',
 'Bag Of Spuds, The',
 'Dublin Streets',
 'A Fish Out Of Water',
 "A Fisherman's Song For Attracting Seals",
 'A French

## azure_language_cognition

In [28]:
import os
import sys
import urllib.request

count = 0
for title in collect_title:
    client_id = "EFVKF10RAlJ1RCn0aXwz"
    client_secret = "x4R27oJktg"
    encQuery = urllib.parse.quote(title)
    data = "query=" + encQuery
    url = "https://openapi.naver.com/v1/papago/detectLangs"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        lang = response_body.decode('utf-8')[13:-2]
        if lang == 'en':
            count += 1
    else:
        print("Error Code:" + rescode)

In [29]:
count

5417

In [34]:
def convert_token(token):
  if isinstance(token, pyabc.Note):
    return f"{token.midi_pitch}//{token.duration}"
  if isinstance(token, pyabc.Rest):
    return f"{0}//{token.duration}"
  text = token._text
  if '"' in text:
    text = text.replace('"', '')
  if text == 'M':
    return None
  if text == '\n':
    return None
  if text in ['u', 'v', '.']:
    return None
  return text

def is_used_token(token):
  if isinstance(token, pyabc.ChordSymbol):
    return False
  return True

def _tune_to_list_of_str(tune):
  return [convert_token(token) for token in tune.tokens if is_used_token(token) and convert_token(token) is not None]


In [35]:
_tune_to_list_of_str(collect_tunes[0])

['55//1.0',
 '52//1.0',
 '52//1.0',
 '52//2.0',
 '50//1.0',
 '52//1.0',
 '50//1.0',
 '48//1.0',
 '|',
 '55//1.0',
 '52//1.0',
 '52//1.0',
 '52//1.0',
 '53//1.0',
 '55//1.0',
 '57//1.0',
 '59//1.0',
 '60//1.0',
 '|',
 '55//1.0',
 '52//1.0',
 '52//1.0',
 '52//2.0',
 '50//1.0',
 '52//1.0',
 '50//1.0',
 '48//1.0',
 '|',
 '57//1.0',
 '50//1.0',
 '50//1.0',
 '55//1.0',
 '52//1.0',
 '48//1.0',
 '50//2.0',
 '57//1.0',
 '|',
 '55//1.0',
 '52//1.0',
 '52//1.0',
 '52//2.0',
 '50//1.0',
 '52//1.0',
 '50//1.0',
 '48//1.0',
 '|',
 '55//1.0',
 '52//1.0',
 '52//1.0',
 '52//1.0',
 '53//1.0',
 '55//1.0',
 '57//1.0',
 '59//1.0',
 '60//1.0',
 '|',
 '55//1.0',
 '52//1.0',
 '52//1.0',
 '52//2.0',
 '50//1.0',
 '52//1.0',
 '50//1.0',
 '48//1.0',
 '|',
 '57//1.0',
 '50//1.0',
 '50//1.0',
 '55//1.0',
 '52//1.0',
 '48//1.0',
 '50//2.0',
 '50//1.0',
 '|',
 '52//1.0',
 '50//1.0',
 '52//1.0',
 '60//2.0',
 '57//1.0',
 '59//1.0',
 '57//1.0',
 '55//1.0',
 '|',
 '52//1.0',
 '50//1.0',
 '52//1.0',
 '57//0.5',
 '59//0.5'